In [68]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch

data = pd.read_csv("../../data/clean_weather.csv")
data = data.ffill()

In [69]:
PREDICTORS = ["tmax", "tmin", "rain"]
TARGET = "tmax_tomorrow"

scaler = StandardScaler()
data[PREDICTORS] = scaler.fit_transform(data[PREDICTORS])

split_data = np.split(data, [int(.7*len(data)), int(.85*len(data))])
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = [[torch.from_numpy(d[PREDICTORS].to_numpy()), torch.from_numpy(d[[TARGET]].to_numpy())] for d in split_data]

In [90]:
import torch

device = torch.device("cpu")

from torch.utils.data import Dataset

class WeatherDataset(Dataset):
    def __init__(self, x, y):
        self.dataset = [x,y]
        self.sequence_length = 10

    def __len__(self):
        return len(self.dataset[0]) - self.sequence_length

    def __getitem__(self, idx):
        x, y = self.dataset[0][idx:(idx+self.sequence_length)], self.dataset[1][idx:(idx+self.sequence_length)]
        return x.float(), y.float()

In [91]:
from torch.utils.data import DataLoader
from torch import nn

BATCH_SIZE = 32

train_dataset = WeatherDataset(train_x, train_y)
valid_dataset = WeatherDataset(valid_x, valid_y)
test_dataset = WeatherDataset(test_x, test_y)

train = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [92]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.rnn = nn.RNN(3, 2, 1, batch_first=True)
        self.dense = nn.Linear(2, 1)

    def forward(self, x):
        x, hidden = self.rnn(x)
        x = self.dense(x)
        return x

In [96]:
from statistics import mean

model = NeuralNetwork().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-4)

EPOCHS = 50
for epoch in range(EPOCHS):
    for batch, (sequence, target) in enumerate(train):
        optimizer.zero_grad()

        sequence = sequence.to(device)
        pred = model(sequence)

        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()

    losses = []
    with torch.no_grad():
        for batch, (sequence, target) in enumerate(valid):
            sequence = sequence.to(device)
            pred = model(sequence)
            loss = loss_fn(pred, target)
            losses.append(loss.item())

    print(f"Epoch {epoch} valid loss: {mean(losses)}")

Epoch 0 valid loss: 952.1530151367188
Epoch 1 valid loss: 241.89063565693203
Epoch 2 valid loss: 105.26522294301836
Epoch 3 valid loss: 75.41163604978532
Epoch 4 valid loss: 67.81001784309508
Epoch 5 valid loss: 64.77374085925874
Epoch 6 valid loss: 54.002420152936665
Epoch 7 valid loss: 42.40914641486274
Epoch 8 valid loss: 36.743278109838094
Epoch 9 valid loss: 33.16698031955295
Epoch 10 valid loss: 30.823495289636035
Epoch 11 valid loss: 29.459445499238512
Epoch 12 valid loss: 28.26100718785846
Epoch 13 valid loss: 27.691665588863312
Epoch 14 valid loss: 26.710173016502743
Epoch 15 valid loss: 25.87721261524019
Epoch 16 valid loss: 25.443804332188197
Epoch 17 valid loss: 25.25991321745373
Epoch 18 valid loss: 25.333948377578977
Epoch 19 valid loss: 26.052796197316002
Epoch 20 valid loss: 24.084434236798966
Epoch 21 valid loss: 23.533674300663055
Epoch 22 valid loss: 23.552133893209792
Epoch 23 valid loss: 23.295174250527033
Epoch 24 valid loss: 23.67743697999016
Epoch 25 valid loss:

In [63]:
pred

tensor([[[64.2513],
         [64.5311],
         [64.5296],
         [64.5305],
         [64.5315],
         [64.5336],
         [64.5341]],

        [[64.3253],
         [64.5336],
         [64.5314],
         [64.5344],
         [64.5364],
         [64.5339],
         [64.5276]],

        [[63.9540],
         [64.5034],
         [64.5137],
         [64.5210],
         [64.5210],
         [64.5220],
         [64.5151]]])